### Step 1. GPU Usage Check ###
If the memory usage below is not 1MiB, it means that GPU-using tasks are already running on this instance.

In [ ]:
!nvidia-smi

### ComfyUI Basic Environment Setup ###
This cell is responsible for installing all the basic programs needed to run ComfyUI and setting up the environment. The script consists of four main parts.

1. Git and Git-LFS Installation

git: A version control tool needed to download ComfyUI's source code from GitHub.

git-lfs: An essential extension program for downloading large files of several gigabytes (GB) such as AI models.

The code checks if these two programs are already installed and automatically proceeds with installation only if they are missing, saving time.

2. ComfyUI Installation

Downloads the latest source code of ComfyUI from GitHub using the git clone command.

The downloaded files are stored in a folder called ComfyUI under the current directory.

3. Essential Python Library Installation

Installs various Python libraries (PyTorch, NumPy, etc.) needed for ComfyUI to function properly.

The requirements.txt file is like an 'installation manual' that lists the required libraries, and installation is performed all at once using this file.

4. ComfyUI-Manager Installation

Installs ComfyUI-Manager, an essential extension for ComfyUI.

Through this manager, you can easily search and install necessary custom nodes or models within the ComfyUI interface with just a few clicks, making it very convenient.

In conclusion, running this cell provides users with a clean basic environment ready to run ComfyUI without going through complex installation processes.

In [ ]:
import os
import subprocess
import sys

def run_command(command, description):
    """Run a command and handle errors"""
    print(f">>> {description}...")
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        if result.stdout:
            print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        if e.stderr:
            print(f"Error details: {e.stderr}")
        return False

# Check and install Git and Git-LFS
print(">>> Checking Git and Git-LFS installation...")
git_check = subprocess.run("git --version", shell=True, capture_output=True)
git_lfs_check = subprocess.run("git lfs version", shell=True, capture_output=True)

if git_check.returncode != 0 or git_lfs_check.returncode != 0:
    print(">>> Git or Git-LFS is not installed. Starting installation.")
    
    # Install GitPython
    run_command("pip install gitpython", "Installing GitPython")
    
    # Install Git-LFS using conda
    run_command("conda install -c conda-forge git-lfs -y", "Installing Git-LFS")
    
    # Initialize Git-LFS
    run_command("git lfs install", "Initializing Git-LFS")
    
    print(">>> Git and Git-LFS installation completed.")
else:
    print(">>> Git and Git-LFS are already installed.")

# Clone ComfyUI
if not os.path.exists('ComfyUI'):
    print(f">>> Cloning ComfyUI to '{os.getcwd()}/ComfyUI'...")
    if run_command("git clone https://github.com/comfyanonymous/ComfyUI.git", "Cloning ComfyUI"):
        print(">>> ComfyUI clone completed.")
    else:
        print(">>> ComfyUI clone failed.")
        sys.exit(1)
else:
    print(">>> ComfyUI directory already exists.")

# Install Python requirements
print(">>> Installing essential Python libraries...")
os.chdir('ComfyUI')
print(os.getcwd())

if run_command("pip install -r requirements.txt", "Installing requirements"):
    print(">>> Python library installation completed.")
else:
    print(">>> Python library installation failed.")

# Install ComfyUI-Manager
os.chdir('custom_nodes')
if not os.path.exists('ComfyUI-Manager'):
    print(">>> Installing ComfyUI-Manager...")
    if run_command("git clone https://github.com/ltdrdata/ComfyUI-Manager.git", "Installing ComfyUI-Manager"):
        print(">>> ComfyUI-Manager installation completed.")
    else:
        print(">>> ComfyUI-Manager installation failed.")
else:
    print(">>> ComfyUI-Manager already exists.")

print(">>> All installations completed successfully!")

### Essential AI Model Download ###
This section downloads the essential AI models needed to run ComfyUI workflows. The models include:

1. **Base Models**: Stable Diffusion checkpoints for image generation
2. **ControlNet Models**: For precise control over image generation
3. **VAE Models**: For encoding/decoding images
4. **Upscaler Models**: For high-resolution image enhancement
5. **IPAdapter Models**: For style transfer and image conditioning

**Note**: Model downloads may take significant time depending on your internet connection speed. Total download size is approximately 15-20GB.

In [ ]:
import os
import urllib.request
from pathlib import Path

def download_file(url, filepath):
    """Download a file from URL to filepath"""
    if os.path.exists(filepath):
        print(f"File already exists: {filepath}")
        return True
    
    try:
        print(f"Downloading: {os.path.basename(filepath)}...")
        urllib.request.urlretrieve(url, filepath)
        print(f"Downloaded: {filepath}")
        return True
    except Exception as e:
        print(f"Error downloading {filepath}: {e}")
        return False

# Change to ComfyUI directory
os.chdir('/home/sagemaker-user/comfyui_on_SageMakerAI/ComfyUI')

# Create model directories
model_dirs = [
    'models/checkpoints',
    'models/controlnet',
    'models/vae',
    'models/upscale_models',
    'models/ipadapter',
    'models/clip_vision'
]

for dir_path in model_dirs:
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    print(f"Created directory: {dir_path}")

# Model download URLs and paths
models_to_download = [
    # Base checkpoint model
    {
        'url': 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors',
        'path': 'models/checkpoints/v1-5-pruned-emaonly.safetensors'
    },
    # VAE model
    {
        'url': 'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors',
        'path': 'models/vae/vae-ft-mse-840000-ema-pruned.safetensors'
    },
    # ControlNet models
    {
        'url': 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth',
        'path': 'models/controlnet/control_v11f1p_sd15_depth.pth'
    },
    {
        'url': 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth',
        'path': 'models/controlnet/control_v11p_sd15_canny.pth'
    },
    # Upscaler model
    {
        'url': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
        'path': 'models/upscale_models/RealESRGAN_x4plus.pth'
    }
]

# Download models
print("Starting model downloads...")
for model in models_to_download:
    download_file(model['url'], model['path'])

print("\n>>> All model downloads completed!")
print(">>> ComfyUI is now ready to use.")